<a href="https://colab.research.google.com/github/artem-dorofeev/py_DataScience_HW/blob/main/HW07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Рекомендаційна система

### Етап 1. Підготовка даних

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163004 sha256=3905f63cc1a11b091d0aee098853b4dbab31233c0721065a97cee61be9555f78
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVDpp
from surprise import NMF

In [5]:
ds_ml100k = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(ds_ml100k, test_size=0.25)

model = SVD()
model.fit(trainset)

predictions = model.test(testset)

accuracy.rmse(predictions)

# У цьому коді ми використовуємо модель SVD для рекомендацій та оцінюємо її точність на тестовій вибірці.

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
RMSE: 0.9441


0.9440944011559843

In [6]:
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.1, 0.2]}

svd = SVD()

grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

grid_search.fit(ds_ml100k)

print("Best parameters found:", grid_search.best_params)
print("Best RMSE score:", grid_search.best_score['rmse'])
print("Best MAE score:", grid_search.best_score['mae'])

# У цьому прикладі ми визначаємо параметри n_epochs, lr_all та reg_all, і GridSearchCV спробує всі можливі комбінації цих параметрів,
# використовуючи крос-валідацію та оцінюючи модель за метриками RMSE та MAE.


Best parameters found: {'rmse': {'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.1}, 'mae': {'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.1}}
Best RMSE score: 0.935783366014447
Best MAE score: 0.7415951600056089


In [7]:
best_params = grid_search.best_params['rmse']

final_model = SVD(n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])

trainset = ds_ml100k.build_full_trainset()
final_model.fit(trainset)

user_id = str(150)
item_id = str(302)
prediction = final_model.predict(user_id, item_id)

print(f"Prediction for user {user_id} and item {item_id}: {prediction.est}")

# З цим кодом ми отримаємо прогнозований рейтинг для конкретного користувача та елемента на основі побудованої моделі.

Prediction for user 150 and item 302: 4.405669356911792


*також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.*

In [8]:
svdpp = SVDpp()


grid_search_svdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search_svdpp.fit(ds_ml100k)

print("Best parameters found for SVD++:", grid_search_svdpp.best_params)
print("Best RMSE score for SVD++:", grid_search_svdpp.best_score['rmse'])
print("Best MAE score for SVD++:", grid_search_svdpp.best_score['mae'])


Best parameters found for SVD++: {'rmse': {'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.02}, 'mae': {'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.02}}
Best RMSE score for SVD++: 0.9279381601976886
Best MAE score for SVD++: 0.7285405494859094


*SVDpp - дуже довге виконання, майже 20хв*

In [9]:
best_params_svdpp = grid_search_svdpp.best_params['rmse']

final_model_svdpp = SVDpp(**best_params_svdpp)

trainset_svdpp = ds_ml100k.build_full_trainset()
final_model_svdpp.fit(trainset_svdpp)

# user_id = str(150)
# item_id = str(302)
prediction_svdpp = final_model_svdpp.predict(user_id, item_id)

print(f"Prediction for user {user_id} and item {item_id}: {prediction_svdpp.est}")


Prediction for user 150 and item 302: 4.44113872154265


In [11]:
nmf = NMF()

param_grid_nmf = {'n_factors': [5, 10, 15], 'n_epochs': [10, 15], 'reg_pu': [0.02, 0.1, 0.2], 'reg_qi': [0.02, 0.1, 0.2]}
grid_search_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=3)
grid_search_nmf.fit(ds_ml100k)


print("Best parameters found for NMF:", grid_search_nmf.best_params)
print("Best RMSE score for NMF:", grid_search_nmf.best_score['rmse'])
print("Best MAE score for NMF:", grid_search_nmf.best_score['mae'])


Best parameters found for NMF: {'rmse': {'n_factors': 10, 'n_epochs': 15, 'reg_pu': 0.1, 'reg_qi': 0.2}, 'mae': {'n_factors': 5, 'n_epochs': 15, 'reg_pu': 0.2, 'reg_qi': 0.2}}
Best RMSE score for NMF: 0.9645511143908981
Best MAE score for NMF: 0.7540425805128521


In [12]:
best_params_nmf = grid_search_nmf.best_params['rmse']

final_model_nmf = NMF(**best_params_nmf)

trainset_nmf = ds_ml100k.build_full_trainset()
final_model_nmf.fit(trainset_nmf)

# user_id = str(150)
# item_id = str(302)
prediction_nmf = final_model_nmf.predict(user_id, item_id)

print(f"Prediction for user {user_id} and item {item_id}: {prediction_nmf.est}")

Prediction for user 150 and item 302: 4.2579098067936


In [13]:
print(f"Prediction svd for user {user_id} and item {item_id}: {prediction.est}, RMSE: {grid_search.best_score['rmse']}")
print(f"Prediction svdpp for user {user_id} and item {item_id}: {prediction_svdpp.est}, RMSE: {grid_search_svdpp.best_score['rmse']}")
print(f"Prediction nmf for user {user_id} and item {item_id}: {prediction_nmf.est}, RMSE: {grid_search_nmf.best_score['rmse']}")

Prediction svd for user 150 and item 302: 4.405669356911792, RMSE: 0.935783366014447
Prediction svdpp for user 150 and item 302: 4.44113872154265, RMSE: 0.9279381601976886
Prediction nmf for user 150 and item 302: 4.2579098067936, RMSE: 0.9645511143908981
